<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow2.0_Eager_Execution_Tutorials/blob/master/tutorials/99_others/probabilistic_model_by_tfp_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q --upgrade tf-nightly-2.0-preview
!pip install -q tfp-nightly

## TFP 基本

In [0]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

tfk = tf.keras
tfkl = tf.keras.layers
tfd = tfp.distributions
tfpl = tfp.layers

### 単回帰
パラメータは $w_\mu, w_\sigma, b_\mu, b_\sigma$ の合計4つ。

$$
p(y\mid x, w_\mu, w_\sigma, b_\mu, b_\sigma) = {\mathcal N} (y \mid w_\mu x + b_\mu,  w_\sigma x+b_\sigma)
$$

In [0]:
input_dim = 1
output_dim = 1

model = tfk.Sequential([
    tfkl.InputLayer([input_dim]),
    tfkl.Dense(tfpl.IndependentNormal.params_size(output_dim)),
    tfpl.IndependentNormal(output_dim)
])

In [4]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 4         
_________________________________________________________________
independent_normal (Independ ((None, 1), (None, 1))    0         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [0]:
X = tf.constant([[3.],
                 [2.],
                 [-5.]])

In [6]:
model(X)

<tfp.distributions.Independent 'Independentsequential/independent_normal/IndependentNormal/Normal/' batch_shape=(3,) event_shape=(1,) dtype=float32>

In [7]:
model(X).sample()

<tf.Tensor: id=236, shape=(3, 1), dtype=float32, numpy=
array([[-1.1187897 ],
       [ 0.34330684],
       [ 4.023557  ]], dtype=float32)>

### 重回帰
入力 $D$ 次元とすれば、$2D+2 = 2(D+1)$ 個のパラメータ（平均、分散を表すための$D$ 次元重みベクトルの2つと、2個のバイアスパラメータ ）を持つ。

$$
p(y\mid {\bf x, w_\mu, w_\sigma}, b_\mu, b_\sigma)
= \mathcal N(y \mid {\bf w^T_\mu x} + b_\mu, {\bf w^T_\sigma x} + b_\sigma)
$$

In [0]:
input_dim = 3
output_dim = 1

model = tfk.Sequential([
    tfkl.InputLayer([input_dim]),
    tfkl.Dense(tfpl.IndependentNormal.params_size(output_dim)),
    tfpl.IndependentNormal(output_dim)
])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 8         
_________________________________________________________________
independent_normal_1 (Indepe ((None, 1), (None, 1))    0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [0]:
X = tf.constant([[3., -4., 1.],
                 [2., 2., 2.],
                 [-5., 3., 1.]])

In [11]:
model(X)

<tfp.distributions.Independent 'Independentsequential_1/independent_normal_1/IndependentNormal/Normal/' batch_shape=(3,) event_shape=(1,) dtype=float32>

In [12]:
model(X).sample()

<tf.Tensor: id=473, shape=(3, 1), dtype=float32, numpy=
array([[-1.7436618],
       [ 3.3002102],
       [ 1.3500538]], dtype=float32)>

### まとめ
例を見ていただいたとおり、`tfp.layers`のモジュールは`tf.keras.layers`と同等に扱えるように実装されています。

ただし、`tf.keras.layers`のモジュールは基本的に`__call__`メソッドが`tf.Tensor`型の入力を受け取り、`tf.Tensor`型を出力します。一方で`tfp.layers`のモジュールはモデリングこそKerasライクにできますが、出力は`tfp.distributions.Distribution`クラス（あるいはそれを継承した子クラス）を表示しているだけで、具体的な`tf.Tensor`の値を得るためには `sample()`メソッドを呼び出す必要があります。

この実装によって、モデルから複数のサンプルを得る必要がある場合は下記のように`sample(n)`。とすることで `n`個のサンプルが得られます。

In [13]:
model(X).sample(5)

<tf.Tensor: id=570, shape=(5, 3, 1), dtype=float32, numpy=
array([[[-1.7289776],
        [ 3.3053372],
        [ 2.9502137]],

       [[-1.7091746],
        [ 3.2955818],
        [-0.8753116]],

       [[-1.7086447],
        [ 3.2720976],
        [ 2.708112 ]],

       [[-1.7175404],
        [ 3.2794054],
        [ 3.0619233]],

       [[-1.7442192],
        [ 3.30328  ],
        [-3.3940215]]], dtype=float32)>

この性質は、ベイズモデルを実装した際に、ベイズ予測分布

$$
p(y\mid x) = \int _ w p(y \mid w, x)p(w\mid D)dw 
$$

をモンテカルロサンプリング

$$
\begin{align}
w_i &\sim p(w \mid D)\\
y_i &\sim p(y\mid w_i, x) \\
\end{align}
$$

で近似する際に便利です。このときできる限り多くのサンプルを使ってヒストグラムを構成することで予測分布を近似できます。これは積分を和で有限近似していることに相当します。

$$
p(y \mid x) \simeq \frac{1}{N} \sum _ {i=1} ^N p(y\mid w_i, x) p(w_i \mid D)
$$

### 変分ベイズ重回帰
統計モデルを

$$
p(y\mid {\bf x, w_\mu, w_\sigma}, b_\mu, b_\sigma)
= \mathcal N(y \mid {\bf w^T_\mu x} + b_\mu, {\bf w^T_\sigma x} + b_\sigma)
$$

とし、事前分布を

$$
\begin{align}
p({\bf w_\mu}) = \mathcal N(0, {\bf \Sigma_{w_\mu}})\\
p({\bf w_\sigma}) = \mathcal N(0, {\bf \Sigma_{w_\sigma}})\\
p(b_\mu) = \mathcal N(0, \sigma_{b_\mu})\\
p(b_\sigma) = \mathcal N(0, \sigma_{b_\sigma})
\end{align}
$$

としたモデルに対して、変分分布 $q({\bf w_\mu}), q({\bf w_\sigma}), q(b_\mu) , q(b_\sigma)$をそれぞれ正規分布（多次元の場合は分散が対角行列）とした変分ベイズ重回帰モデルは、入力 $D$ 次元とすれば、$4D + 2$ 個の（変分）パラメータを持ちます。

In [0]:
input_dim = 3
output_dim = 1

model = tfk.Sequential([
    tfkl.InputLayer([input_dim]),
    tfpl.DenseReparameterization(tfpl.IndependentNormal.params_size(output_dim)),
    tfpl.IndependentNormal(output_dim)
])

In [24]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_reparameterization_2 ( (None, 2)                 14        
_________________________________________________________________
independent_normal_3 (Indepe ((None, 1), (None, 1))    0         
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [0]:
X = tf.constant([[3., -4., 1.],
                 [2., 2., 2.],
                 [-5., 3., 1.]])

In [26]:
model(X)

<tfp.distributions.Independent 'Independentindependent_normal_3_1/IndependentNormal/Normal/' batch_shape=(3,) event_shape=(1,) dtype=float32>

In [27]:
model(X).sample()

<tf.Tensor 'Reshape_23:0' shape=(3, 1) dtype=float32>